# Uzavřené provozovny v roce 2021

Jak se zavíraly hospody nebo obchody, odkud jídlo nechcete, v roce 2021.

Zdrojem dat je webová aplikace [Potraviny na pranýři](https://www.potravinynapranyri.cz/), která vzniká na základě šetření Státní zemědělské a potravinářeské inspekce.

In [1]:
import datetime

import altair as alt
import pandas as pd

In [2]:
actual = pd.read_csv('../data/actual.csv', index_col='id', dtype={'id': int, 'ico': str, 'druh': 'category'}, parse_dates=['datum_zverejneni', 'datum_uzavreni', 'datum_uvolneni_zakazu'])
archive = pd.read_csv('../data/archive.csv', index_col='id', dtype={'id': int, 'ico': str, 'druh': 'category'}, parse_dates=['datum_zverejneni', 'datum_uzavreni', 'datum_uvolneni_zakazu'])

In [3]:
print(f"""
Nejstarší záznam v actual.csv je z {actual['datum_zverejneni'].min().date().isoformat()} a nejnovější z {actual['datum_zverejneni'].max().date().isoformat()}.
Nejstarší záznam v archive.csv je z {archive['datum_zverejneni'].min().date().isoformat()} a nejnovější z {archive['datum_zverejneni'].max().date().isoformat()}.
""")


Nejstarší záznam v actual.csv je z 2021-07-08 a nejnovější z 2022-01-07.
Nejstarší záznam v archive.csv je z 2015-03-29 a nejnovější z 2021-06-29.



In [4]:
# Combine actual and archiv datasets
combined = actual.append(archive)
combined.head()

,referencni_cislo,ico,nazev,adresa,datum_zverejneni,datum_uzavreni,datum_uvolneni_zakazu,stav_uzavreni,druh,zjistene_skutecnosti
id,,,,,,,,,,
51052,P22-000004-SZPI-CZ,19014325,"H R U Š K A , spol. s r.o.","Vysoká nad Labem 186, 503 31 Vysoká nad Labem",2022-01-07,2022-01-06,2022-01-07,Znovu otevřeno,Maloobchod,Výskyt trusu hlodavců|Stopy po působení a výsk...
51051,P22-000003-SZPI-CZ,74456571,NAM BUI HUY,"Velké Hamry 492, 468 45 Velké Hamry",2022-01-06,2022-01-05,NaT,Uzavřeno,Maloobchod,Výskyt trusu hlodavců
51050,P22-000002-SZPI-CZ,00031747,"COOP Hořovice, družstvo","Radouš 83, 267 24 Hostomice",2022-01-05,2022-01-04,2022-01-05,Znovu otevřeno,Maloobchod,Výskyt trusu hlodavců|Potraviny poškozené nebo...
51049,P22-000001-SZPI-CZ,28329147,KORČIÁN s.r.o.,671 51 Vranovská Ves 124,2022-01-04,2022-01-04,NaT,Uzavřeno,Rychlé občerstvení,Výskyt trusu hlodavců|Výskyt živých hlodavců|V...
51048,P21-000095-SZPI-CZ,00031747,"COOP Hořovice, družstvo","Kuštova 1769, 269 01 Rakovník",2021-12-22,2021-12-21,2021-12-22,Znovu otevřeno,Maloobchod,Výskyt trusu hlodavců


## Rok 2021

Ohlédnutí za uzavřenými provozovnami v roce 2021.

In [5]:
closed_by_year = (pd.DataFrame(combined.groupby(combined['datum_uzavreni'].dt.year)['referencni_cislo'].count())).reset_index() # Altair does not work with index
closed_by_year.columns = ['rok_uzavreni', 'pocet']
closed_by_year


,rok_uzavreni,pocet
0,2015,221
1,2016,174
2,2017,157
3,2018,177
4,2019,119
5,2020,109
6,2021,94
7,2022,4


In [6]:
# Try Altair chart
bars = alt.Chart(closed_by_year).mark_bar().encode(
    alt.Y('rok_uzavreni:N', title='Rok uzavření'),
    alt.X('pocet:Q', title='Počet'),
    ).properties(title='Počty uzavření v jednotlivých letech')

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='pocet:Q'
)

bars + text

alt.LayerChart(...)

In [7]:
# Pro zjednodušení beru datum uzavření
last_year = combined[
    (combined['datum_uzavreni'] >= datetime.datetime(2021,1,1)) &
    (combined['datum_uzavreni'] <= datetime.datetime(2021,12,31))
    ]

### Druhy provozoven

In [8]:
# Druhy provozoven v seznamu
business_types = last_year['druh'].value_counts().reset_index()
business_types.columns = ['druh', 'pocet']
business_types

,druh,pocet
0,Maloobchod,60
1,Restaurace,11
2,Výroba,10
3,Velkoobchod,7
4,Rychlé občerstvení,3
5,"Pivnice, bar nebo herna",1
6,Čerpací stanice,1
7,Výroba a rozvoz jídel,1


In [9]:
# Try Altair chart
bars = alt.Chart(business_types).mark_bar().encode(
    alt.Y('druh:N', sort='-x', title='Druh'),
    alt.X('pocet:Q', title='Počet'),
    ).properties(title='Druhy uzavřených podniků')

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='pocet:Q'
)

bars + text


alt.LayerChart(...)

## Co se zanedbalo

In [10]:
offenses = last_year['zjistene_skutecnosti'].str.get_dummies()
offenses_freq = offenses.mean()
sorted_offenses_perc = pd.DataFrame((offenses_freq.sort_values(ascending=False) * 100).round(1))
sorted_offenses_perc.index.name = ('Co se nepovedlo')
sorted_offenses_perc.columns = ['Jak často (%)']
sorted_offenses_perc['Jak často (%)'] = sorted_offenses_perc['Jak často (%)'].astype(str).str.replace('.', ',', regex=False) + ' %'

print(sorted_offenses_perc[:10])

                                                Jak často (%)
Co se nepovedlo                                              
Výskyt trusu hlodavců                                  76,6 %
Nečistoty na podlaze                                   52,1 %
Výrazně zanedbaný úklid                                44,7 %
Provozovna nezabezpečená před pronikáním škůdců        41,5 %
Nečistoty na regálech či zařízeních                    30,9 %
Stopy po působení a výskytu hlodavců                   27,7 %
Výskyt mrtvých hlodavců                                23,4 %
Výrazné znečištění prostor                             21,3 %
Výskyt plísní na stěnách                               17,0 %
Poškození stěn                                         16,0 %
